# UnitaryHack 2024 Demo Notebook

In [24]:
# Import the necessary libraries
import os
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_ionq import IonQProvider, GPIGate, GPI2Gate, MSGate, ZZGate 


In [25]:
# Initialize the IonQ provider and backend
api_key = os.getenv("IONQ_API_KEY") or input("Enter your IonQ API key: ")
provider = IonQProvider(token=api_key)
backend = provider.get_backend("simulator", gateset="native")

In [ ]:
# Define a QIS-gate circuit
qc = QuantumCircuit(2, name="bell state")
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Plot the circuit
qc.draw(output="mpl")

In [ ]:
# Transpile the circuit to the native gate set
# Here, you will need to override the Qiskit optimizer
# to use the IonQ optimizer. The goal is to outperform the
# Qiskit optimizer in terms of gate count and circuit depth.
t_qc = transpile(qc, backend=backend, optimization_level=3, pass_manager = pm)

# Plot the transpiled circuit
t_qc.draw(output="mpl")

In [ ]:
# Run the circuit
job = backend.run(t_qc, noise_model="aria-1", shots=1000)

In [ ]:
# Get the result
print(job.get_counts())

# Plot the result
plot_histogram(job.get_counts())

In [ ]:
from qiskit.visualization import plot_gate_map
plot_gate_map(backend)

In [ ]:
print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
)

In [ ]:
backend_config = backend.configuration()

In [ ]:
print(backend.gateset())

In [ ]:
# Access detailed gate properties
print(backend_config.basis_gates)

In [75]:
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.dagcircuit import DAGCircuit

class GPI2Cancellation(TransformationPass):
    def run(self, dag: DAGCircuit) -> DAGCircuit:
        nodes_to_remove = []

        # Traverse all nodes in the DAG
        for node in dag.topological_op_nodes():
            if node.name == 'gpi2':
                successors = list(dag.successors(node))  # Convert the iterator to a list
                print(successors)
                for next_node in successors:
                    if next_node.name == 'gpi2':
                        # Check if they cancel each other out
                        phi1 = node.op.params[0]
                        phi2 = next_node.op.params[0]
                        if (phi1 + 0.5) % 1 == phi2 % 1 or (phi2 + 0.5) % 1 == phi1 % 1:
                            # Mark both nodes for removal
                            nodes_to_remove.extend([node, next_node])

        # Remove the marked nodes
        for node in nodes_to_remove:
            if dag.has_node(node):  # Check if the node is still in the DAG
                dag.remove_op_node(node)
        
        return dag


In [71]:
from qiskit.transpiler import PassManager
from qiskit.converters import circuit_to_dag, dag_to_circuit

pm = PassManager([GPI2Cancellation()])

In [78]:
qc = QuantumCircuit(2, name="2cnots")
qc.cx(0, 1)
qc.cx(0, 1)

optimized_circuit = pm.run(qc)
ibm_transpiled = transpile(qc, backend=backend, optimization_level=3)

In [74]:
a = pm.run(ibm_transpiled)

AttributeError: 'DAGOutNode' object has no attribute 'name'